# Databricks

https://databricks.com

In [0]:
# !pip install pyspark

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import array
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

In [0]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier

In [0]:
# Some functions that convert our CSV input data into numerical
# features for each job candidate
def binary(YN): # To be completed by Student
    if (YN == 'yes' or YN == 'Yes'):
        return 1
    else:
        return 0

def mapSex(degree): # To be completed by Student
    if (degree == 'f'):
        return 1
    else:
        return 0

def mapEthnicity(ethnic): # To be completed by Student 
    if(ethnic == 'asian'):
        return 1
    else:
        return 0

def mapAssessor(assessor): # To be completed by Student 
    if(assessor == 'Health Care Professional'):
        return 1
    else:
        return 0


In [0]:
# Convert a list of raw fields from our CSV file to a
# LabeledPoint that MLLib can use. All data must be numerical...
# def createLabeledPoints(fields):
#     ageMonths = int(fields[0])
#     sex = mapSex(fields[1])
#     Ethnicity = mapEthnicity(fields[2])
#     jaundice = binary(fields[3])
#     hereditary = binary(fields[4])
#     asd = binary(fields[5])
     

#     return LabeledPoint(asd, array([ageMonths,sex,
#         Ethnicity, jaundice, hereditary]))

  
def createLabeledPoints(fields):
    A1 = int(fields[0])
    A2 = int(fields[1])
    A3 = int(fields[2])
    A4 = int(fields[3])
    A5 = int(fields[4])
    A6 = int(fields[5])
    A7 = int(fields[6])
    A8 = int(fields[7])
    A9 = int(fields[8])
    A10 = int(fields[9])
    ageMonths = int(fields[10])
    sex = mapSex(fields[11])
    Ethnicity = mapEthnicity(fields[12])
    jaundice = binary(fields[13])
    hereditary = binary(fields[14])
    Assessor = mapAssessor(fields[15])
    asd = binary(fields[16])

    return LabeledPoint(asd, array([A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, ageMonths, sex,
        Ethnicity, jaundice, hereditary, Assessor]))


In [0]:
#Load up our CSV file, and filter out the header line with the column names
rawData = sc.textFile("/FileStore/tables/Toddler_Autism_dataset_BigData.csv")
header = rawData.first()
rawData = rawData.filter(lambda x:x != header)


In [0]:
# Split each line into a list based on the comma delimiters
csvData = rawData.map(lambda x: x.split(","))

In [0]:
# Convert these lists to LabeledPoints
trainingData = csvData.map(createLabeledPoints)

In [0]:
# # Train our DecisionTree classifier using our data set
# model = DecisionTree.trainClassifier(trainingData, numClasses=2,
#                                      categoricalFeaturesInfo={1:2, 2:6, 3:2, 4:2},
#                                      impurity='gini', maxDepth=5, maxBins=32)

model = DecisionTree.trainClassifier(trainingData, numClasses=2,
                                     categoricalFeaturesInfo={11:2, 12:6, 13:2, 14:2, 15:4},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [0]:
# Create a test candidate
# testCandidates = [ array([36, 0, 2, 1, 0])] # test different candidates
testCandidates = [ array([1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 36, 0, 2, 1, 0, 1])] # test different candidates
testData = sc.parallelize(testCandidates)

In [0]:
# Now get predictions (Note, you could separate
# the source data into a training set and a test set while tuning
# parameters and measure accuracy as you go!)
predictions = model.predict(testData)
print('ASD prediction:')
results = predictions.collect()
for result in results:
    print(result)

ASD prediction:
1.0

In [0]:
# We can also print out the decision tree itself:
print('Learned classification tree model:')
print(model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 5 with 35 nodes
 If (feature 8 <= 0.5)
 If (feature 6 <= 0.5)
 If (feature 7 <= 0.5)
 If (feature 3 <= 0.5)
 Predict: 0.0
 Else (feature 3 > 0.5)
 If (feature 9 <= 0.5)
 Predict: 0.0
 Else (feature 9 > 0.5)
 Predict: 1.0
 Else (feature 7 > 0.5)
 If (feature 5 <= 0.5)
 If (feature 1 <= 0.5)
 Predict: 0.0
 Else (feature 1 > 0.5)
 Predict: 1.0
 Else (feature 5 > 0.5)
 Predict: 1.0
 Else (feature 6 > 0.5)
 If (feature 1 <= 0.5)
 If (feature 4 <= 0.5)
 If (feature 0 <= 0.5)
 Predict: 0.0
 Else (feature 0 > 0.5)
 Predict: 1.0
 Else (feature 4 > 0.5)
 Predict: 1.0
 Else (feature 1 > 0.5)
 Predict: 1.0
 Else (feature 8 > 0.5)
 If (feature 5 <= 0.5)
 If (feature 6 <= 0.5)
 If (feature 7 <= 0.5)
 If (feature 1 <= 0.5)
 Predict: 0.0
 Else (feature 1 > 0.5)
 Predict: 1.0
 Else (feature 7 > 0.5)
 Predict: 1.0
 Else (feature 6 > 0.5)
 Predict: 1.0
 Else (feature 5 > 0.5)
 If (feature 4 <= 0.5)
 If (feature 10 <= 13.5)
 If (feature 1 <= 0.5)
 Predict: 0.0
 Else (feature 1 > 0.5)
 Predict: 1.0
 Else (feature 10 > 13.5)
 Predict: 1.0
 Else (feature 4 > 0.5)
 Predict: 1.0

---
`The end is also called the new start.` --- ISS : **I** **S**(elf) **S**(tudy)

---